In [ ]:
!pip install datasets
!pip install transformers


     |████████████████████████████████| 298 kB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 63.2 MB/s 
     |████████████████████████████████| 243 kB 76.9 MB/s 
     |████████████████████████████████| 61 kB 399 kB/s 
     |████████████████████████████████| 132 kB 75.8 MB/s 
     |████████████████████████████████| 271 kB 75.5 MB/s 
     |████████████████████████████████| 192 kB 70.6 MB/s 
     |████████████████████████████████| 160 kB 60.8 MB/s 
     |████████████████████████████████| 3.3 MB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 57.0 MB/s 
     |████████████████████████████████| 596 kB 68.7 MB/s 
     |████████████████████████████████| 895 kB 74.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from transformers import Wav2Vec2Processor, HubertForCTC
from datasets import load_dataset
import soundfile as sf
import time
import string
import tensorflow as tf

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
from transformers import Wav2Vec2Processor, HubertForCTC
from datasets import load_dataset
import torch

dataset = load_dataset("librispeech_asr", "other", split="test")
sampling_rate = dataset.features["audio"].sampling_rate




Downloading:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/329M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/30.6G [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset librispeech_asr downloaded and prepared to /root/.cache/huggingface/datasets/librispeech_asr/other/2.1.0/f4f37a3069fb71dc8d2984a4647ed7e0ff909bc9f0f9a183550cbc4e2db847ed. Subsequent calls will reuse this data.


In [ ]:
dataset

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 2939
})

In [ ]:
data =dataset.remove_columns([ "speaker_id"])

In [ ]:
processor = Wav2Vec2Processor.from_pretrained('facebook/hubert-large-ls960-ft')
model = HubertForCTC.from_pretrained('facebook/hubert-large-ls960-ft')


Downloading:   0%|          | 0.00/212 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/138 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

In [ ]:
type(dataset)

datasets.arrow_dataset.Dataset

In [ ]:
def map_to_array(batch):
    speech, _ = sf.read(batch["text"])
    batch["speech"] = speech
    return batch

In [ ]:
dataset[0]['audio']

{'array': array([-9.15527344e-05,  1.22070312e-04,  3.05175781e-05, ...,
         4.88281250e-04,  2.44140625e-04,  2.44140625e-04]),
 'path': '7902-96591-0000.flac',
 'sampling_rate': 16000}

In [ ]:
dataset[0]["audio"]["array"]

array([-9.15527344e-05,  1.22070312e-04,  3.05175781e-05, ...,
        4.88281250e-04,  2.44140625e-04,  2.44140625e-04])

In [ ]:
dataset[0]["text"]

'I AM FROM THE CUTTER LYING OFF THE COAST'

In [ ]:
len(dataset)

2939

In [ ]:
ds = []
ts = [] 
for i in range(len(dataset)):
  ds.append(dataset[i]["audio"]["array"])
  ts.append(dataset[i]["text"])

In [ ]:
ds

[array([-9.15527344e-05,  1.22070312e-04,  3.05175781e-05, ...,
         4.88281250e-04,  2.44140625e-04,  2.44140625e-04]),
 array([ 2.44140625e-04,  7.01904297e-04,  7.32421875e-04, ...,
        -6.10351562e-05, -1.52587891e-04, -9.15527344e-05]),
 array([-6.10351562e-05, -1.83105469e-04, -1.22070312e-04, ...,
         3.35693359e-04,  1.52587891e-04,  5.18798828e-04]),
 array([ 3.05175781e-05,  2.44140625e-04,  9.15527344e-05, ...,
        -3.05175781e-05, -1.22070312e-04, -2.13623047e-04]),
 array([-1.52587891e-04, -1.83105469e-04, -2.44140625e-04, ...,
         1.22070312e-04, -6.10351562e-05,  9.15527344e-05]),
 array([-3.05175781e-05, -3.05175781e-05, -3.05175781e-05, ...,
        -1.83105469e-04, -9.15527344e-05, -9.15527344e-05]),
 array([-9.15527344e-05,  0.00000000e+00,  0.00000000e+00, ...,
        -1.52587891e-04, -9.15527344e-05, -1.83105469e-04]),
 array([ 1.22070312e-04,  6.10351562e-05, -3.05175781e-05, ...,
         0.00000000e+00,  6.10351562e-05,  0.00000000e+00]),


In [ ]:
transcript=[]
for i in range(len(ds)):
  input_values = processor(ds[i],sampling_rate=16000,padding = True, return_tensors="pt").input_values  # Batch size 1
  logits = model(input_values).logits
  predicted_ids = torch.argmax(logits, dim=-1)
  transcription = processor.decode(predicted_ids[0])
  transcript.append(transcription)

In [ ]:
len(transcript)

2939

In [ ]:
transcript[:200]

["I'M FROM THE CUTTER LYING OFF THE COAST",
 "DON'T CRY HE SAID I WAS OBLIGED TO COME",
 'AND AND YOU HAVE NOT FOUND OUT ANYTHING CAME IN QUICK FRIGHTENED TONES',
 'I WISH YOU WOULD BELIEVE ME THAT I AM IN AS GREAT TROUBLE ABOUT IT AS YOU ARE',
 'THAT MY FATHER SIR RISDON GRAHAM HAS SMUGGLED GOODS HERE',
 'HE COULD NOT HELP IT HE HATES THE SMUGGLERS YOU SHALL NOT TELL',
 'PRAY PRAY SAY YOU WILL NOT ARCHIE WAS SILENT',
 'THEN AS ARCHIE STOOD IN THE DARK LITERALLY AGHAST WITH ASTONISHMENT HE HEARD THE FAINT RUSTLING ONCE MORE AND AGAIN ALL WAS SILENT',
 'HE LAUGHED BUT IT WAS A CURIOUS KIND OF LAUGH FULL OF VEXATION INJURED AMOR PROPRE AS THE FRENCH CALL OUR LOVE OF OUR OWN DIGNITY OF WHICH ARCHIBALD RAYSTROKE IN THE FULL FLUSH OF HIS YOUNG BELIEF IN HIS IMPORTANCE AS A BRITISH OFFICER HAD A PRETTY GOOD STOCK',
 'IT ALL COMES OF DRESSING UP IN THIS STUPID WAY LIKE A ROUGH FISHER LAD',
 'COLD WATER CAME ON THIS IDEA DIRECTLY AS HE RECALLED THE FACT THAT THE DARKNESS WAS INTENSE AND CELIA 

In [ ]:
target=dataset["text"]

In [ ]:
len(target)

2939

In [ ]:
len(dataset)


2939

In [ ]:
!pip install jiwer
import jiwer
from jiwer import wer


In [ ]:
print(wer(transcript,target))

0.04229630337185978
